<a href="https://colab.research.google.com/github/YogeshRathee512/Machine-learning-projects/blob/main/learning_nano_gpt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

--2025-01-25 07:35:28--  https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.111.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1115394 (1.1M) [text/plain]
Saving to: ‘input.txt’

input.txt           100%[===================>]   1.06M  --.-KB/s    in 0.06s   

2025-01-25 07:35:28 (17.1 MB/s) - ‘input.txt’ saved [1115394/1115394]



In [2]:
with open("input.txt", "r",encoding ='utf-8') as f:
  text = f.read()

In [5]:
len(text)

1115394

In [7]:
list(set(text))

['M',
 'z',
 'a',
 ';',
 'O',
 '.',
 'B',
 'r',
 'g',
 'b',
 ',',
 ' ',
 'A',
 'x',
 'Q',
 'Z',
 '!',
 'D',
 'k',
 '&',
 'e',
 'W',
 'G',
 'n',
 'N',
 'o',
 'I',
 '$',
 'u',
 'H',
 'J',
 'X',
 'C',
 'l',
 ':',
 "'",
 'P',
 'K',
 'd',
 'h',
 'Y',
 'L',
 't',
 'T',
 'w',
 'f',
 '-',
 'i',
 '\n',
 'y',
 'F',
 '3',
 'm',
 '?',
 'R',
 'p',
 'U',
 'q',
 'S',
 'V',
 's',
 'c',
 'E',
 'j',
 'v']

In [12]:
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(''.join(chars))
print(vocab_size)


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
65


In [13]:
stoi = {ch:i for i,ch in enumerate(chars)}

In [15]:
itos = {i:ch for i,ch in enumerate(chars)}

In [19]:
encode = lambda s : [stoi[c] for c in s]
decode = lambda l : ''.join([itos[i] for i in l])

In [20]:
print(encode("hii there"))

[46, 47, 47, 1, 58, 46, 43, 56, 43]


In [23]:
import torch
data  = torch.tensor(encode(text),dtype = torch.long)
print(data.shape,data.dtype)
print(data[:10])

torch.Size([1115394]) torch.int64
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47])


In [24]:
n = int(0.9*len(data))
train_data = data[:n]
val_data = data[n:]

In [25]:
block_size = 8
train_data[:block_size+1]

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58])

In [44]:
x = train_data[:block_size]
y = train_data[1:block_size+1]

In [45]:
print(y)

tensor([47, 56, 57, 58,  1, 15, 47, 58])


In [46]:
print(x)

tensor([18, 47, 56, 57, 58,  1, 15, 47])


In [47]:
for t in range(block_size):
  context = x[:t+1]
  target = y[t]
  print(f"when input is {context} the target : {target}")

when input is tensor([18]) the target : 47
when input is tensor([18, 47]) the target : 56
when input is tensor([18, 47, 56]) the target : 57
when input is tensor([18, 47, 56, 57]) the target : 58
when input is tensor([18, 47, 56, 57, 58]) the target : 1
when input is tensor([18, 47, 56, 57, 58,  1]) the target : 15
when input is tensor([18, 47, 56, 57, 58,  1, 15]) the target : 47
when input is tensor([18, 47, 56, 57, 58,  1, 15, 47]) the target : 58


In [58]:
torch.manual_seed(1337)
batch_size = 4
block_size = 8

def get_batch(split):
  data = train_data if split == 'train' else val_data
  ix  = torch.randint(len(data)-block_size,(batch_size,))
  x = torch.stack([data[i:i+block_size] for i in ix])
  y = torch.stack([data[i+1:i+block_size+1] for i in ix])
  return x,y

In [59]:
xb,yb = get_batch('train')

In [65]:
print(xb)
print(xb.shape)
print(" ")
print('targets:')
print(yb.shape)
print(yb)
print("----")

tensor([[24, 43, 58,  5, 57,  1, 46, 43],
        [44, 53, 56,  1, 58, 46, 39, 58],
        [52, 58,  1, 58, 46, 39, 58,  1],
        [25, 17, 27, 10,  0, 21,  1, 54]])
torch.Size([4, 8])
 
targets:
torch.Size([4, 8])
tensor([[43, 58,  5, 57,  1, 46, 43, 39],
        [53, 56,  1, 58, 46, 39, 58,  1],
        [58,  1, 58, 46, 39, 58,  1, 46],
        [17, 27, 10,  0, 21,  1, 54, 39]])
----


In [66]:
for b in range(batch_size):
  for t in range(block_size):
    context = xb[b, :t+1]
    target = yb[b,t]


In [67]:
import torch
import torch.nn as nn

from torch.nn import functional as F
torch.manual_seed(1337)

class BigramLanguageModel(nn.Model):

  def __init__(self,vocab_size):
    super().__init__()
    self.token_embedding_table = nn.Embedding(vocab_size,vocab_size)

  def forward(self,idx,target):
    logits = self.token_embedding_table(idx)

    return logits

m = BigramLanguageModel(vocab_size)
out = m(xb,yb)
print(out.shape)